In [ ]:
import pandas as pd
train = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_train.csv',encoding='ISO-8859-1')
train.head()

In [ ]:
train=train[['OriginalTweet','Sentiment']]
train.head()

In [ ]:
train.Sentiment.value_counts()

In [ ]:
train.replace({'Sentiment' : { 'Extremely Negative' :0, 'Negative' : 0, 'Neutral' : 'N',
                              'Extremely Positive' :1, 'Positive' : 1, }},inplace=True)
train.Sentiment.value_counts()

In [ ]:
train = train[train.Sentiment != 'N']
train.Sentiment.value_counts()

In [ ]:
test = pd.read_csv('../input/covid-19-nlp-text-classification/Corona_NLP_test.csv')
test.head()

In [ ]:
test=test[['OriginalTweet','Sentiment']]
test.head()

In [ ]:
test.replace({'Sentiment' : { 'Extremely Negative' :0, 'Negative' : 0, 'Neutral' : "N",
                              'Extremely Positive' :1, 'Positive' : 1, }},inplace=True)
test.Sentiment.value_counts()

In [ ]:
test = test[test.Sentiment != 'N']
test.Sentiment.value_counts()

In [ ]:
X_train=list(train['OriginalTweet'])
X_train[:5]

In [ ]:
X_test=list(test['OriginalTweet'])
X_test[:5]

In [ ]:
y_train=list(train['Sentiment'])
y_train[:5]

In [ ]:
y_test=list(test['Sentiment'])
y_test[:5]

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(X_train, truncation=True, padding=True)
test_encodings = tokenizer(X_test, truncation=True, padding=True)

In [ ]:
import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
))

In [ ]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
)

In [ ]:
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = TFTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=test_dataset             # evaluation dataset
)

trainer.train()

In [ ]:
trainer.evaluate(test_dataset)

In [ ]:
trainer.predict(test_dataset)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,trainer.predict(test_dataset)[1],target_names=['Negative','Positive']))